In [1]:
!pip install gliner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 109.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 106.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [4]:
# ============================================================
# GLiNER Zero-shot NER Evaluation (CROSSNER – TEST ONLY)
# - Test split only
# - Batched inference → manual micro-F1
# - Fixed random seed (reproducible)
# ============================================================

from pathlib import Path
import pandas as pd
from tqdm import tqdm
import torch
import random
import numpy as np

from gliner import GLiNER


# ============================================================
# 0. RANDOM SEED (REPRODUCIBLE)
# ============================================================

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# ============================================================
# 1. CONFIG
# ============================================================

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
THRESHOLD = 0.5
BATCH_SIZE = 32

CROSSNER_ROOT = "/kaggle/input/crossner/ner_data"
CROSSNER_SUBSETS = ["ai", "literature", "music", "politics", "science"]

MODELS = {
    "GLiNER-L": "urchade/gliner_large-v2.1",
}


# ============================================================
# 2. BIO → SPANS
# ============================================================

def bio_to_spans(tokens, tags):
    spans = []
    start, label = None, None

    for i, t in enumerate(tags):
        if t.startswith("B-"):
            if label is not None:
                spans.append((start, i - 1, label))
            start = i
            label = t[2:]
        elif t.startswith("I-"):
            continue
        else:
            if label is not None:
                spans.append((start, i - 1, label))
                start, label = None, None

    if label is not None:
        spans.append((start, len(tokens) - 1, label))

    return spans


# ============================================================
# 3. LOAD CROSSNER (TEST ONLY)
# ============================================================

def load_crossner_test(subset):
    path = Path(CROSSNER_ROOT) / subset / "test.txt"
    data, tokens, tags = [], [], []

    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            if not line:
                if tokens:
                    data.append({
                        "tokens": tokens,
                        "spans": bio_to_spans(tokens, tags)
                    })
                    tokens, tags = [], []
                continue

            tok, tag = line.split()
            tokens.append(tok)
            tags.append(tag)

    return data


# ============================================================
# 4. CHAR → TOKEN ALIGNMENT
# ============================================================

def token_boundaries(tokens):
    bounds, pos = [], 0
    for t in tokens:
        bounds.append((pos, pos + len(t)))
        pos += len(t) + 1
    return bounds


def char_to_token(preds, boundaries):
    out = set()

    for p in preds:
        cs, ce, label = p["start"], p["end"], p["label"]
        s = e = None

        for i, (a, b) in enumerate(boundaries):
            if a <= cs < b:
                s = i
            if a < ce <= b:
                e = i

        if s is not None and e is not None and s <= e:
            out.add((s, e, label))

    return out


# ============================================================
# 5. BATCH INFERENCE (STORE ALL PREDICTIONS)
# ============================================================

def run_inference(model, dataset, all_labels, desc):
    texts = [" ".join(ex["tokens"]) for ex in dataset]
    boundaries = [token_boundaries(ex["tokens"]) for ex in dataset]

    all_preds = []

    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc=desc):
        batch_texts = texts[i:i + BATCH_SIZE]
        batch_bounds = boundaries[i:i + BATCH_SIZE]

        preds_batch = model.inference(
            batch_texts,
            all_labels,
            threshold=THRESHOLD,
            flat_ner=True,
            batch_size=BATCH_SIZE,
        )

        for preds, bounds in zip(preds_batch, batch_bounds):
            all_preds.append(char_to_token(preds, bounds))

    return all_preds


# ============================================================
# 6. MANUAL MICRO-F1 (EXACT MATCH)
# ============================================================

def compute_f1(dataset, preds):
    tp = fp = fn = 0

    for ex, pred in zip(dataset, preds):
        gold = set(ex["spans"])

        tp += len(gold & pred)
        fp += len(pred - gold)
        fn += len(gold - pred)

    p = tp / (tp + fp) if tp + fp > 0 else 0
    r = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * p * r / (p + r) if p + r > 0 else 0

    return round(f1 * 100, 1)


# ============================================================
# 7. LOAD DATASETS + LABEL SETS (TEST ONLY)
# ============================================================

datasets = {
    sub.capitalize(): load_crossner_test(sub)
    for sub in CROSSNER_SUBSETS
}

label_sets = {
    name: sorted({l for ex in ds for _, _, l in ex["spans"]})
    for name, ds in datasets.items()
}


# ============================================================
# 8. RUN ALL MODELS
# ============================================================

rows = []

for model_name, model_path in MODELS.items():
    print(f"\n🚀 Loading {model_name}")
    model = GLiNER.from_pretrained(model_path).to(DEVICE)

    row = {"Model": model_name}
    scores = []

    for dname, dset in datasets.items():
        preds = run_inference(
            model,
            dset,
            label_sets[dname],
            desc=f"{model_name} | {dname}"
        )

        f1 = compute_f1(dset, preds)
        row[dname] = f1
        scores.append(f1)

    row["Average"] = round(sum(scores) / len(scores), 1)
    rows.append(row)


# ============================================================
# 9. RESULT TABLE
# ============================================================

df = pd.DataFrame(rows)
df = df[
    ["Model", "Ai", "Literature", "Music", "Politics", "Science", "Average"]
]

display(df)



🚀 Loading GLiNER-L


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
GLiNER-L | Ai:   0%|          | 0/14 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
GLiNER-L | Ai: 100%|██████████| 14/14 [00:06<00:00,  2.18it/s]
GLiNER-L | Literature: 100%|██████████| 13/13 [00:06<00:00,  2.04it/s]
GLiNER-L | Music: 100%|██████████| 15/15 [00:07<00:00,  2.01it/s]
GLiNER-L | Politics: 100%|██████████| 21/21 [00:09<00:00,  2.19it/s]
GLiNER-L | Science: 100%|██████████| 17

,Model,Ai,Literature,Music,Politics,Science,Average
0,GLiNER-L,52.9,59.0,71.5,65.4,61.0,62.0
